In [1]:
import numpy as np

## Case 1: Orange and Bag

In [2]:
price_orange = 1.5
count_orange = 3
price_bag = 0.5

In [3]:
def f1(price_orange, count_orange, price_bag):
    return price_orange * count_orange + price_bag

In [4]:
f1(price_orange, count_orange, price_bag)

5.0

## AddLayer MulLayer

In [48]:
class AddLayer():
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        return dout, dout

In [49]:
class MulLayer():
    def __init(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

## Solve Case 1 using Layers

In [50]:
orange_l = MulLayer()
orange_v = orange_l.forward(price_orange, count_orange)

In [51]:
bag_l = AddLayer()
bag_v = bag_l.forward(orange_v, price_bag)

In [52]:
bag_v

5.0

In [54]:
d_orange, d_price_bag = bag_l.backward(1)

In [55]:
d_price_orange, d_count_orange = orange_l.backward(d_orange)

In [56]:
print("d_price_orange: ", d_price_orange)
print("d_count_orange: ", d_count_orange)
print("d_price_bag: ", d_price_bag)

d_price_orange:  3
d_count_orange:  1.5
d_price_bag:  1


In [57]:
f1(price_orange, count_orange, price_bag)

5.0

In [58]:
f1(price_orange+1, count_orange, price_bag)

8.0

In [59]:
f1(price_orange, count_orange+1, price_bag)

6.5

In [60]:
f1(price_orange, count_orange, price_bag+1)

6.0

## Case 2: Smoothie

In [92]:
price_orange = 1.5
price_mango = 2.5
count_smoothie = 3
price_bag = 0.5

In [93]:
def f2(price_orange, price_mango, count_smoothie, price_bag):
    return price_orange * count_smoothie + price_mango * count_smoothie + price_bag

In [94]:
f2(price_orange, price_mango, count_smoothie, price_bag)

12.5

## Solve Case 2 using Layers

In [95]:
l_orange = MulLayer()
v_orange = l_orange.forward(price_orange, count_smoothie)

l_mango = MulLayer()
v_mango = l_mango.forward(price_mango, count_smoothie)

In [96]:
l_smoothie = AddLayer()
v_smoothie = l_smoothie.forward(v_orange, v_mango)

In [97]:
l_total = AddLayer()
v_total = l_total.forward(v_smoothie, price_bag)

In [98]:
v_total

12.5

In [99]:
d_smoothie, d_price_bag = l_total.backward(1)

In [100]:
d_orange, d_mango = l_smoothie.backward(d_smoothie)

In [101]:
d_price_mango, d_count_mango = l_mango.backward(d_mango)

In [102]:
d_price_orange, d_count_orange = l_orange.backward(d_orange)

In [103]:
d_count_smoothie = d_count_orange + d_count_mango

In [104]:
print("d_price_orange: ", d_price_orange)
print("d_price_mango: ", d_price_mango)
print("d_count_smoothie: ", d_count_smoothie)
print("d_price_bag: ", d_price_bag)

d_price_orange:  3
d_price_mango:  3
d_count_smoothie:  4.0
d_price_bag:  1


In [105]:
f2(price_orange, price_mango, count_smoothie, price_bag)

12.5

In [106]:
f2(price_orange+1, price_mango, count_smoothie, price_bag)

15.5

In [107]:
f2(price_orange, price_mango+1, count_smoothie, price_bag)

15.5

In [108]:
f2(price_orange, price_mango, count_smoothie+1, price_bag)

16.5

In [109]:
f2(price_orange, price_mango, count_smoothie, price_bag+1)

13.5

## Case 3: Fruits in CA, NY, and TX

$$
\boldsymbol{C} = \begin{bmatrix}
    c_{orange} & c_{mango}
\end{bmatrix}
$$

$$
\boldsymbol{P} = \begin{bmatrix}
    p_{orange, CA} & p_{orange, NY} & p_{orange, TX} \\
    p_{mango, CA}  & p_{mango, NY}  & p_{mango, TX}  \\
\end{bmatrix}
$$

$$
\boldsymbol{b} = \begin{pmatrix}
    b_{CA} & b_{NY} & b_{TX}
\end{pmatrix}
$$

$$
Total = \sum C P + b
$$

In [110]:
C = np.array([
    [3, 2]
])
P = np.array([
    [1.5, 1.0, 2.0],
    [3.0, 2.0, 4.0]
])
b = np.array([0.5, 1.0, 0.7])

In [112]:
np.dot(C, P) + b

array([[ 11. ,   8. ,  14.7]])

In [113]:
np.sum(np.dot(C, P) + b)

33.700000000000003

In [124]:
def f3(C, P, b):
    return C[0,0] * P[0,0] + C[0,1] * P[1,0] + C[0,0] * P[0,1] + C[0,1] * P[1,1] +  C[0,0] * P[0,2] + C[0,1] * P[1,2] + b[0] + b[1] + b[2]

In [125]:
f3(C, P, b)

33.700000000000003

## MatMul and MatAdd Layers

In [149]:
class MatMul:
    def __init__(self):
        self.X = None
        self.W = None
        
    def forward(self, X, W):
        Y = np.dot(X, W)
        self.X = X
        self.W = W
        return Y
    
    def backward(self, dY):
        dX = np.dot(dY, self.W.T)
        dW = np.dot(self.X.T, dY)
        return dX, dW

In [150]:
class MatAdd:
    def forward(self, A, b):
        return A + b
    
    def backward(self, dY):
        dA = dY
        db = np.sum(dY, axis=0)
        return dA, db

In [151]:
l_fruit = MatMul()
v_fruit = l_fruit.forward(C, P)

In [152]:
l_bag = MatAdd()
v_bag = l_bag.forward(v_fruit, b)

In [153]:
np.sum(v_bag)

33.700000000000003

In [154]:
d_v_fruit, d_b = l_bag.backward(np.ones((1, 3)))

In [155]:
dC, dP = l_fruit.backward(d_v_fruit)

In [157]:
print("dC:")
print(dC)
print("dP:")
print(dP)
print("d_b:")
print(d_b)

dC:
[[ 4.5  9. ]]
dP:
[[ 3.  3.  3.]
 [ 2.  2.  2.]]
d_b:
[ 1.  1.  1.]


In [158]:
C = np.array([
    [3, 2]
])
P = np.array([
    [1.5, 1.0, 2.0],
    [3.0, 2.0, 4.0]
])
b = np.array([0.5, 1.0, 0.7])
f3(C, P, b)

33.700000000000003

In [160]:
# Increase mango count
C2 = np.array([
    [3, 3]
])
P2 = np.array([
    [1.5, 1.0, 2.0],
    [3.0, 2.0, 4.0]
])
b2 = np.array([0.5, 1.0, 0.7])
f3(C2, P2, b2)

42.700000000000003

In [161]:
C2 = np.array([
    [3, 2]
])
# Decrease orange price in TX
P2 = np.array([
    [1.5, 1.0, 1.0],
    [3.0, 2.0, 4.0]
])
b2 = np.array([0.5, 1.0, 0.7])
f3(C2, P2, b2)

30.699999999999999

## Case 4: Group of friends in CA, NY, and TX

In [173]:
C = np.array([
    [3, 2],
    [1, 0],
    [0, 4],
    [5, 6],
])

In [174]:
np.sum(np.dot(C, P) + b)

157.30000000000001

In [175]:
l_fruit = MatMul()
v_fruit = l_fruit.forward(C, P)
l_bag = MatAdd()
v_bag = l_bag.forward(v_fruit, b)

d_v_fruit, d_b = l_bag.backward(np.ones((4, 3)))
dC, dP = l_fruit.backward(d_v_fruit)

In [176]:
print("dC:")
print(dC)
print("dP:")
print(dP)
print("d_b:")
print(d_b)

dC:
[[ 4.5  9. ]
 [ 4.5  9. ]
 [ 4.5  9. ]
 [ 4.5  9. ]]
dP:
[[  9.   9.   9.]
 [ 12.  12.  12.]]
d_b:
[ 4.  4.  4.]


In [177]:
# Original
C = np.array([
    [3, 2],
    [1, 0],
    [0, 4],
    [5, 6],
])
P = np.array([
    [1.5, 1.0, 2.0],
    [3.0, 2.0, 4.0]
])
b = np.array([0.5, 1.0, 0.7])
np.sum(np.dot(C, P) + b)

157.30000000000001

In [178]:
# CA bag price increase
C2 = np.array([
    [3, 2],
    [1, 0],
    [0, 4],
    [5, 6],
])
P2 = np.array([
    [1.5, 1.0, 2.0],
    [3.0, 2.0, 4.0]
])
b2 = np.array([1.5, 1.0, 0.7])
np.sum(np.dot(C2, P2) + b2)

161.30000000000001

In [179]:
# Original
C2 = np.array([
    [3, 3],
    [1, 0],
    [0, 4],
    [5, 6],
])
P2 = np.array([
    [1.5, 1.0, 2.0],
    [3.0, 2.0, 4.0]
])
b2 = np.array([0.5, 1.0, 0.7])
np.sum(np.dot(C2, P2) + b2)

166.30000000000001